In [1]:
import keras
from keras import layers

## 필터 개수가 변경되는 잔차 블록

In [2]:
inputs = keras.Input(shape=(32, 32, 3))

x = layers.Conv2D(32, 3, activation='relu')(inputs)
residual = x # 잔차를 따로 저장

# 잔차 블록에 해당하는 층
# 출력 필터를 32개에서 64개로 증가
# 패딩으로 인해 다운샘플링이 되지 않도록 padding='same'으로 지정
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
# 잔차는 32개의 필터만 있으므로 1 * 1 Conv2D를 사용하여 적절한 크기로 투영
residual = layers.Conv2D(64, 1)(residual)
# 이제 블록 출력과 잔차의 크기가 같으므로 더할 수 있음
x = layers.add([x, residual])

## 최대 풀링 층을 가진 잔차 블록

In [3]:
inputs = keras.Input(shape=(32, 32, 3))

x = layers.Conv2D(32, 3, activation='relu')(inputs)
residual = x # 잔차를 따로 저장

# 이 잔차 블록은 2 * 2 최대 풀링 층을 포함하여 2개의 층으로 구성
# 패딩으로 인해 다운샘플링이 되지 않도록 합성곱 층과 최대 풀링 층에 padding='same'을 지정
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2, padding='same')(x)

# 최대 풀링 층으로 인한 다운샘플링에 맟추기 위해 잔차 투영에 strides=2를 사용
residual = layers.Conv2D(64, 1, strides=2)(residual)
# 이제 블록 출력과 잔차의 크기가 같으므로 더할 수 있음
x = layers.add([x, residual])

## 여러 개의 블록으로 구성된 간단한 컨브넷의 예

In [4]:
def residual_block(x, filters, pooling=False) :
    residual = x
    x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
    x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
    if pooling :
        x = layers.MaxPooling2D(2, padding='same')(x)
        # 최대 풀링을 사용 시 스트라이드 합성곱을 추가
        residual = layers.Conv2D(filters, 1, strides=2)(residual)
    # 최대 풀링을 사용하지 않을 시 채널 수가 바뀐 경우에만 잔차를 투영
    elif filters != residual.shape[-1]: 
        residual = layers.Conv2D(filters, 1)(residual)
    x = layers.add([x, residual])
    return x



In [5]:
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Rescaling(1./255)(inputs)

# 첫 번째 블록
x = residual_block(x, filters=32, pooling=True)
# 두 번째 블록, 블록마다 필터 개수가 증가
x = residual_block(x, filters=64, pooling=True)
# 마지막 블록은 바로 다음에 전역 평균 풀링(global average pooling)을 사용하기 때문에 최대 풀링이 필요하지 않음
x = residual_block(x, filters=128, pooling=False)

x = layers.GlobalAveragePooling2D()(x) # 전체를 평균으로 pooling
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 32, 32, 3)    0           ['input_3[0][0]']                
                                                                                                  
 conv2d_6 (Conv2D)              (None, 32, 32, 32)   896         ['rescaling[0][0]']              
                                                                                                  
 conv2d_7 (Conv2D)              (None, 32, 32, 32)   9248        ['conv2d_6[0][0]']               
                                                                                              